In [1]:
import os
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ruptures as rpt
from __future__ import division
from more_itertools import chunked
%matplotlib widget

In [2]:
# name example: 20200304-160513 10pN- 30nM DNAp + 150nM_gp2.5 + 0.1mg_mL_BSA #019-002
name = input('please type in the file name:' ) + '.xlsx'
# path example: C:\Users\KTS260\Desktop
path = input('please type in the file path:')
filename = os.path.join(path, name) 

please type in the file name: 20191017-015-2
please type in the file path: C:\Users\86183\Google Cloud\Work with Matthew\Watching labelled DNAP replication\20191017-015-2-exo+pol -good +5mM Mg2+


In [ ]:
# For simple use, we inport data in a excel file
sheet_name = "Sheet1"
data =  pd.read_excel(filename,sheet_name = sheet_name)
time = data['time']/1000
time = np.array(time)
basepairs = data['basepairs'] # 1D array
basepairs = np.array(basepairs)
plt.figure(figsize = (6,6))
plt.plot(time, basepairs)
plt.xlabel('Time/s')
plt.ylabel('Basepairs')
plt.title('Example Basepairs-Time')
#plt.plot(dbphat10_1_dt)
plt.show()

In [6]:
# please enter the time interval of interest (ROI)
time_from = float(input('please type in the starting time of interest:'))
time_to = float(input('please type in the ending time of interest:'))

please type in the starting time of interest: 30
please type in the ending time of interest: 42


In [7]:
# here you will select and analyze your interesting cycle with corresponding time range; 
# Or you will analyze the whole trace at one witm

# define a temporary index to compute time of ROI, and subsuquent ROI of force, distance
indtemp = np.where((time <= time_to) & (time >= time_from))

time_ROI = time[indtemp]
basepairs_ROI = basepairs[indtemp]
# basepairs = basepairs_ROI

In [8]:
# From now on we work on change point detection
signal = basepairs_ROI  # 1D array

# Apply proper model & searching method to work on change point detection
model = "l2"  # "l1", "rbf", "linear", "normal", "ar",...

# For faster predictions, one can modify the jump parameter during initialization. The higher it is, the faster the prediction is achieved (at the expense of precision).
# There are plenty of models can be used for change point detection; but only rpt.Pelt seems to work with a algo.predict(pen=)
algo = rpt.Pelt(model=model, jump=1).fit(signal)
# algo = rpt.BottomUp(model=model).fit(signal)
# algo = rpt.Window(width=40, model=model).fit(signal)
# algo = rpt.Binseg(model=model).fit(signal)
# algo = rpt.Dynp(model="rbf", params=params, jump=1, min_size=2).fit(signal)


# In our case, the number of change points is unknown, we need to specify a threshold on the residual norm using epsilon or a penalty using the pen parameter.
# n = number of samples
# sigma = noise standard deviation; the bigger the sigma, the less break points
# dim = dimention
n = len(signal)
sigma = 20
# my_bkps = algo.predict(epsilon=3 * n * sigma ** 2)
# # or
dim = 1
my_bkps = algo.predict(pen=np.log(n) * dim * sigma ** 2)
## or you can assign a breakpoint number, e.g.:
# my_bkps = algo.predict(n_bkps=3)

# show results
fig, (ax,) = rpt.display(signal, my_bkps, figsize=(8, 4))

plt.xlabel('Data Index')
plt.ylabel('Basepairs')
plt.title('Rupture Detection')
plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# my_bkps returns index of break points
# trace back to time-basepair based on index
time_bkp = np.zeros(len(my_bkps)-1)
bps_bkp = np.zeros(len(my_bkps)-1)

# to exclude the last value
time_bkp = time_ROI[my_bkps[0:-1]]
bps_bkp = basepairs_ROI[my_bkps[0:-1]]

# for ind in range(len(my_bkps)-1):
#     time_bkp[ind] = time_ROI[my_bkps[ind]]
#     bps_bkp[ind] = basepairs_ROI[my_bkps[ind]]

# to add the first value and last value to array
time_bkp = np.insert(time_bkp,0,time_ROI[0])
time_bkp = np.append(time_bkp,time_ROI[-1])
bps_bkp = np.insert(bps_bkp,0,basepairs_ROI[0])
bps_bkp = np.append(bps_bkp,basepairs_ROI[-1])
# bks = np.append(time_bkp,bps_bkp, axis=-1).reshape((2,len(my_bkps)+1)).T


In [10]:
duration_time = np.diff(time_bkp)
processivity_event = np.diff(bps_bkp)
speed = processivity_event / duration_time 
print(duration_time)
print(processivity_event)
print(speed)

[2.148 2.5   0.4   0.55  0.4   1.    0.298 0.351 0.65  0.301 0.249 0.4
 0.6   0.601 0.598 0.249 0.55  0.102]
[-47.59853 -35.31519 -39.8978  -36.92262 -36.5224  -46.94765 -53.93152
 -65.63654 -51.52807 -62.21937 -60.00722 -27.74859 -37.02377 -52.52199
 -22.77016 -56.6978  -50.24387  -6.42095]
[ -22.15946462  -14.126076    -99.7445      -67.13203636  -91.306
  -46.94765    -180.97825503 -186.99868946  -79.27395385 -206.70887043
 -240.99285141  -69.371475    -61.70628333  -87.39099834  -38.07719064
 -227.70200803  -91.35249091  -62.9504902 ]


In [11]:
excel_filename = filename + '-' + sheet_name + str(time_from) + '-' +str(time_to) + '.xlsx'

writer = pd.ExcelWriter(excel_filename)
data_1 = {'breaking time/s':time_bkp,
        'breaking position/bp':bps_bkp}
data_2 = {'event duration/s':duration_time,
        'event processivity/bp':processivity_event,
        'velocity/(bp/s)':speed}
df_1= pd.DataFrame(data_1)
df_2 = pd.DataFrame(data_2)

df_1.to_excel(writer,sheet_name ='breaking points', index=False)
df_2.to_excel(writer,sheet_name ='final data', index=False)

writer.save()

In [12]:
from scipy import stats
sem = stats.sem(speed, axis=None, ddof=0)
avg = np.mean(speed)

print(avg)
print(sem)

-104.16218242222136
16.39283650442368


In [13]:
fig, ax = plt.subplots(figsize=(6, 4))
n_bins = 30
n, bins, patches = ax.hist(speed, n_bins)
ax.set_title('Pol Rate')
ax.set_xlabel('Pol Rate(bp/s)')
ax.set_ylabel('Occurrence')
plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
plt.figure(figsize=(10,6))
# plt.plot(time_10pN/1000,basepairs_10pN)
plt.plot(time_ROI, basepairs_ROI)
plt.scatter(time_bkp,bps_bkp, marker = "^",s=50,color='red')
plt.xlabel('Time/s')
plt.ylabel('Basepairs')
plt.title('Example Rupture Detection')
plt.tight_layout()

# plt.savefig("C:\\Users\\KTS260\\Desktop\\' + name + '-cycle#'+ cycle + '.png",dpi=None, facecolor='w', edgecolor='w',
#         orientation='portrait',format=None,
#         transparent=False, bbox_inches=None, pad_inches=0.1,
#         metadata=None)
# plt.savefig(cycledic + '.png')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …